In [1]:
import pandas as pd
import json
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
raw_tweets_text=[]

def get_data(url):
    with open(url) as fp:
        for line in fp:
            tweet_as_dict=json.loads(line)
            raw_tweets_text.append(tweet_as_dict['text'])
    convert_to_csv()

def convert_to_csv():    
    df=pd.DataFrame(raw_tweets_text)
    df.to_csv('tweets.csv')

In [3]:
get_data("tweets_file.txt")

KeyError: 'text'

In [4]:
d = pd.read_csv("tweets.csv")
d = d.drop("Unnamed: 0",axis=1)
d = d.rename(columns={"0":"Text"})
d.head(10)

,Text
0,RT @SarkySage: As my account grows it’s more d...
1,RT @theseoulstory: B.I releases a lyric video ...
2,@7stargalaxy I genuinely can't even imagine fo...
3,"RT @minunivers: ""If there’s a name i call you ..."
4,Scary af. And plausible.
5,Track and Trace are utterly useless. The proof...
6,We camped out because we NEVER lost faith in t...
7,RT @possum_bimbo: Should I do it? https://t.co...
8,RT @Deepak11412544: BHPV merged with BHEL on 3...
9,RT @maria_aat6: I checked the results from the...


In [5]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6307 entries, 0 to 6306
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    6307 non-null   object
dtypes: object(1)
memory usage: 49.4+ KB


In [6]:
def process_data(df):
    text_emoji = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text_links = re.compile('httpstco[a-zA-Z0-9]+')
    text_rts = re.compile('RT [A-Za-z0-9]+')
    text_schar = re.compile('[^A-Za-z0-9 ]+')
    text_nums = re.compile('[0-9]+')
    
    for i in range(len(df)):
        txt=df["Text"].iloc[i]
        txt=text_emoji.sub(r'',txt)
        temp=txt.translate(txt.maketrans('','',string.punctuation))
        temp=text_links.sub(r'',temp)
        temp=text_rts.sub(r'',temp)        
        temp=text_schar.sub(r'',temp)
        temp=text_nums.sub(r'',temp)
        df["Text"].iloc[i] = temp.lower()

In [7]:
process_data(d)
d.head(10)

,Text
0,as my account grows its more difficult to kee...
1,bi releases a lyric video for his collab song...
2,stargalaxy i genuinely cant even imagine for a...
3,if theres a name i call you by should i call ...
4,scary af and plausible
5,track and trace are utterly useless the proof ...
6,we camped out because we never lost faith in t...
7,should i do it
8,bhpv merged with bhel on this i am not sayin...
9,i checked the results from the last two weeks...


In [ ]:
def further_ops(df):

    for i in range(len(df)):
        tweet = df["Text"].iloc[i]
        token = word_tokenize(tweet)
        token_no_sw = [word for word in token if word not in stopwords.words()]
        lemmatizer = WordNetLemmatizer()
        lemm_token = [lemmatizer.lemmatize(word,'v') for word in token_no_sw]
        df["Text"].iloc[i] = " ".join(lemm_token)

In [ ]:
further_ops(d)
d.head(10)

In [ ]:
index = d[d["Text"]==""].index
d.drop(index,inplace=True)

In [ ]:
d = d.reset_index()

In [ ]:
d.info()

In [ ]:
d.to_csv('processed_tweets.csv')
d.head()